In [1]:
import spacy
import random
import os
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [2]:
anno1 = json.load(open('annotations.json',))
anno2 = json.load(open('annotations2.json',))
anno3 = json.load(open('annotations3.json',))

In [3]:
doc_bin = DocBin() # create a DocBin object

In [3]:
all_annotations = []
all_annotations.extend(anno1['annotations'])
all_annotations.extend(anno2['annotations'])
all_annotations.extend(anno3['annotations'])

In [5]:
all_annotations[1]

['MPS  MORE  202004141732  917316  BANGALORE',
 {'entities': [[0, 3, 'TYPE'],
   [5, 9, 'ID'],
   [11, 23, 'UTR'],
   [25, 31, 'UTR']]}]

In [28]:
from spacy.util import filter_spans

for training_example  in tqdm(all_annotations[:200]): 
    text = training_example[0]
    labels = training_example[1]['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 6600.63it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [37]:
!python3 -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [38]:
!python3 -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-28 13:23:47,856] [INFO] Set up nlp object from config
[2022-12-28 13:23:47,861] [INFO] Pipeline: ['tok2vec', 'parser', 'ner', 'tagger']
[2022-12-28 13:23:47,863] [INFO] Created vocabulary
Traceback (most recent call last):
  File "/home/pranav/UPI_ner/ner/lib/python3.8/site-packages/spacy/language.py", line 1301, in initialize
    init_vocab(
  File "/home/pranav/UPI_ner/ner/lib/python3.8/site-packages/spacy/training/initialize.py", line 131, in init_vocab
    load_vectors_into_model(nlp, vectors)
  File "/home/pranav/UPI_ner/ner/lib/python3.8/site-packages/spacy/training/initialize.py", line 152, in load_vectors_into_model
    vectors_nlp = load_model(name, vocab=nlp.vocab, exclude=exclude)
  File "/home/pranav/UPI_ner/ner/lib/python3.8/site-packages/spacy/util.py", line 439, in load_model
    raise IOError(Errors.E050.format(name=name))
OSError: [E050] C

In [ ]:
nlp_ner = spacy.load("experiment_14-01-23/model-last")

doc = nlp_ner(all_annotations[:200][4][0])

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [11]:
#for j in all_annotations[:200]:
doc = nlp_ner('UPI-SONY PICTURES NETWOR-SONYPICTURESNETWORKSINDIA.RZP@AXISBANK-UTIB0000100-230694811795-UPI INTENT'.lower().replace('-',' '))
colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [8]:
for index in range(len(all_annotations)):
    text,entity = all_annotations[index] 
    doc = nlp_ner(all_annotations[index][0])
    prediction = [(ent.text, ent.label_) for ent in doc.ents]
    final_entitites = []
    for i in entity['entities']:
        final_entitites.append(convert_to_tuple(text,i))
    if final_entitites != prediction:
        print(final_entitites,'|',prediction)
        print('-'*50)

[('UPI', 'TYPE'), ('134717243602', 'UTR'), ('UPI', 'TYPE'), ('aman07mishra', 'ID'), ('StateBankOfI', 'BANK')] | [('UPI', 'TYPE'), ('134717243602', 'UTR'), ('UPI', 'TYPE'), ('StateBankOfI', 'BANK')]
--------------------------------------------------
[('MPS', 'TYPE'), ('MORE', 'ID'), ('201912151759  387876', 'UTR')] | [('MPS', 'TYPE'), ('MORE', 'ID'), ('201912151759', 'REMARK'), ('387876', 'REMARK')]
--------------------------------------------------
[('UPI', 'TYPE'), ('110519236732', 'UTR'), ('UberRide', 'REMARK'), ('uberrides@hdfcb', 'ID')] | [('UPI', 'TYPE'), ('110519236732', 'UTR'), ('UberRide', 'REMARK'), ('uberrides@hdfcb', 'ID'), ('HDFCBAN', 'BANK')]
--------------------------------------------------
[('UPI', 'TYPE'), ('035008466313', 'UTR'), ('UPI', 'TYPE'), ('paytmqr28100505', 'ID'), ('PaytmPayment', 'REMARK')] | [('UPI', 'TYPE'), ('035008466313', 'UTR'), ('UPI', 'TYPE'), ('paytmqr28100505', 'ID'), ('PaytmPayment', 'BANK')]
--------------------------------------------------
[('U

[('INF', 'TYPE'), ('INFT', 'TYPE'), ('60471000002G', 'UTR'), ('SalaryForJun', 'REMARK'), ('ABEERESEARC', 'ID')] | [('INF', 'TYPE'), ('INFT', 'TYPE'), ('60471000002', 'UTR'), ('G', 'REMARK'), ('SalaryForJun', 'REMARK'), ('ABEERESEARC', 'ID')]
--------------------------------------------------
[('UPI', 'TYPE'), ('012254058522', 'UTR'), ('PaymentfromPh', 'REMARK'), ('Q56128298@ybl', 'ID')] | [('UPI', 'TYPE'), ('012254058522', 'UTR'), ('PaymentfromPh', 'REMARK'), ('Q56128298@ybl', 'ID'), ('HDF', 'BANK')]
--------------------------------------------------
[('UPI', 'TYPE'), ('214911384559', 'UTR'), ('Momrecharge', 'REMARK'), ('vilpredel@paytm', 'ID'), ('PaytmPayments', 'BANK')] | [('UPI', 'TYPE'), ('214911384559', 'UTR'), ('Momrecharge', 'REMARK'), ('vilpredel@paytm', 'ID')]
--------------------------------------------------
[('UPI', 'TYPE'), ('920022831320', 'UTR'), ('UPI', 'TYPE'), ('q75503461@ybl', 'ID'), ('AxisBankLtd', 'BANK')] | [('UPI', 'TYPE'), ('920022831320', 'UTR'), ('UPI', 'TYPE'

In [6]:
def convert_to_tuple(text,entity):
    #print(text,entity)
    start,end,typer = entity
    return (text[start:end],typer)

In [9]:
final_entitites

[('UPI', 'TYPE'),
 ('035810194943', 'UTR'),
 ('CollectRequest', 'REMARK'),
 ('vedix@abfspay', 'ID')]